# 1. Generate a CRF Predicted Result Output

In [1]:
import sys
rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner/'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')
from loaddata import loadData, getTestData

import pickle
import random
import pandas as pd

# 2017-10-23
# 2017-11-03
p = rootPath + 'pkldata/CCKS-2018-04-04/CCT_Dict.p'
cctTrain, cctTest = loadData(p, 10)

[len(i) for i in cctTest]

[75, 75, 75, 75]

Here we first generate a test file.

Based on this test file, we could predict its entity tags with a pre-trained model, say, '`1abdp`'.

The test file's path is `'demo/Evals/test.txt'`.

In [2]:
filePath = 'demo/Evals/test.txt'

cols = ['_atom', 'basicTag', 'dictTag', 'POSTag']

getTestData(cctTest, cols, filePath)

---

Here we assume that we have a model named '`1abdp`'

And a test data. Its path is `'demo/Evals/test.txt'`.

---

In [3]:
from crftools import crf_test
# root = 'Users/floyd/Desktop/Research/NER-CRF/cctner/'
Input = filePath
Output = 'demo/Evals/result.txt'

import time
testPath = rootPath + 'crftools/crf_test'
inputPath = Input
outputPath = Output
modelPath = rootPath + 'models/1abdp/model'

btime = time.clock()
crf_test(crf_test_path=testPath,
         modelpath=modelPath,
         testfilepath=inputPath,
         resultpath=outputPath)
etime = time.clock()

print('Time Consumption: %03f s' % (etime - btime))

Time Consumption: 0.011059 s


---

Here, we have a new file: `'demo/Evals/result.txt'`.

Let's have a look!

---

In [4]:
with open(Output, 'r') as f:
    n = 0
    for i in f.readlines():
        print(i)
        n = n + 1
        if n == 7:
            break

# 0.940402

胀	CHN	OTHER	a-B	O/0.967902	Bo-B/0.000438	Bo-I/0.001098	Ch-B/0.000383	Ch-I/0.001133	DT-B/0.000791	DT-I/0.000913	O/0.967902	Si-B/0.002825	Si-I/0.000178	Sy-B/0.012163	Sy-I/0.009039	Tr-B/0.001030	Tr-I/0.001077	unct-B/0.000451	unct-I/0.000579

伴	CHN	BREAK_UNIT	v-B	O/0.979707	Bo-B/0.000004	Bo-I/0.000022	Ch-B/0.000004	Ch-I/0.000020	DT-B/0.000002	DT-I/0.000023	O/0.979707	Si-B/0.000005	Si-I/0.000061	Sy-B/0.000602	Sy-I/0.019318	Tr-B/0.000019	Tr-I/0.000173	unct-B/0.000003	unct-I/0.000039

活	CHN	OTHER	vn-B	Sy-B/0.956382	Bo-B/0.000038	Bo-I/0.000004	Ch-B/0.000299	Ch-I/0.000103	DT-B/0.000047	DT-I/0.000074	O/0.015909	Si-B/0.000116	Si-I/0.000029	Sy-B/0.956382	Sy-I/0.026368	Tr-B/0.000287	Tr-I/0.000239	unct-B/0.000044	unct-I/0.000061

动	CHN	OTHER	vn-I	Sy-I/0.985070	Bo-B/0.000002	Bo-I/0.000021	Ch-B/0.000018	Ch-I/0.000348	DT-B/0.000005	DT-I/0.000011	O/0.013429	Si-B/0.000004	Si-I/0.000015	Sy-B/0.000833	Sy-I/0.985070	Tr-B/0.000088	Tr-I/0.000142	unct-B/0.000003	unct-I/0.000011

受	CHN	OTHER	v-B	Sy-

# 2. Deal With the Result of the Output

In [5]:
from splitresult import splitResult
results = splitResult(Output)           ; print(Output) ; print(len(results))
cctTestList = sum(cctTest, [])          ; print(len(cctTestList))
cols                                    ; print(cols)

demo/Evals/result.txt
300
300
['_atom', 'basicTag', 'dictTag', 'POSTag']


In [6]:
assert len(cctTestList) == len(results)
for idx in range(len(cctTestList)):
    cct    = cctTestList[idx]
    result = results[idx]
    cct.outputCols = cols 
    cct.corpResult(result=result[1], newTag= 'LearnedETag') ## TODO


In [7]:
cct = cctTestList[100]
print(cct.dfFormat.shape)
cct.dfFormat.iloc[40:53]

(292, 8)


,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag,LearnedETag
40,部,CHN,SITE_UNIT,阝,n-I,R-I,Bo-I,Bo-I
41,",",PUNC,OTHER,-,x-B,O,O,O
42,当,CHN,OTHER,彐,t-B,O,O,O
43,时,CHN,OTHER,日,t-I,O,O,O
44,腰,CHN,PART_UNIT,月,n-B,R-B,Bo-B,Sy-B
45,部,CHN,SITE_UNIT,阝,n-I,R-I,Bo-I,Sy-I
46,活,CHN,OTHER,氵,vn-B,O,O,Sy-I
47,动,CHN,OTHER,力,vn-I,O,O,Sy-I
48,受,CHN,OTHER,又,v-B,O,O,Sy-I
49,限,CHN,OTHER,阝,v-I,O,O,Sy-I


In [11]:
cct.annotedEntities[:10]

[('腰部', 39, 40, 'Bo'),
 ('腰部', 44, 45, 'Bo'),
 ('咳嗽', 55, 56, 'Sy'),
 ('疼痛', 71, 72, 'Sy'),
 ('腰痛', 84, 85, 'Sy'),
 ('左下肢', 87, 89, 'Bo'),
 ('疼痛', 90, 91, 'Sy'),
 ('L3-S1椎间盘CT', 97, 106, 'Ch'),
 ('腰椎', 111, 112, 'Bo'),
 ('椎体边缘', 122, 125, 'Bo')]

In [12]:
cct.getLearnedEntities("LearnedETag")
cct.learnedEntities[:10]

[('腰部', 39, 40, 'Bo'),
 ('腰部活动受限', 44, 49, 'Sy'),
 ('腰痛', 84, 85, 'Sy'),
 ('左下肢疼痛', 87, 91, 'Sy'),
 ('腰椎', 111, 112, 'Bo'),
 ('椎体边缘呈刺样增生', 122, 130, 'Sy'),
 ('腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘', 132, 156, 'Sy'),
 ('水平硬膜囊受压', 160, 166, 'Sy'),
 ('T', 175, 175, 'Ch'),
 ('P', 183, 183, 'Ch')]

In [13]:
for cct in cctTestList:
    cct.getLearnedEntities('LearnedETag')

In [14]:
cct = cctTestList[100]
print(cct.originalFilePath)
print(len(cct.annotedEntities)); print(len(cct.learnedEntities))

/Users/floyd/Desktop/Research/NER-CRF/cctner/dataset/annoted/batch1/病史特点/病史特点-193.txtoriginal.txt
25
18


In [15]:
print('Only in Learned: ', [i for i in cct.learnedEntities if i not in cct.annotedEntities])
print('Only in Annoted: ', [i for i in cct.annotedEntities if i not in cct.learnedEntities])
print(cct._text)

Only in Learned:  [('腰部活动受限', 44, 49, 'Sy'), ('左下肢疼痛', 87, 91, 'Sy'), ('椎体边缘呈刺样增生', 122, 130, 'Sy'), ('腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘', 132, 156, 'Sy'), ('水平硬膜囊受压', 160, 166, 'Sy'), ('心肺腹', 218, 220, 'Bo'), ('异常', 223, 224, 'Sy'), ('S1椎体棘突两侧轻压痛', 247, 257, 'Sy'), ('放射', 260, 261, 'Sy'), ('双侧直腿抬高试验', 263, 270, 'Ch'), ('拾物试验', 276, 279, 'Ch')]
Only in Annoted:  [('腰部', 44, 45, 'Bo'), ('咳嗽', 55, 56, 'Sy'), ('疼痛', 71, 72, 'Sy'), ('左下肢', 87, 89, 'Bo'), ('疼痛', 90, 91, 'Sy'), ('L3-S1椎间盘CT', 97, 106, 'Ch'), ('椎体边缘', 122, 125, 'Bo'), ('腰4-腰5、腰5-骶1椎间盘', 132, 145, 'Bo'), ('椎体边缘', 153, 156, 'Bo'), ('硬膜囊', 162, 164, 'Bo'), ('查体', 172, 173, 'Ch'), ('R', 191, 191, 'Ch'), ('心', 218, 218, 'Bo'), ('肺', 219, 219, 'Bo'), ('腹', 220, 220, 'Bo'), ('腰椎', 233, 234, 'Bo'), ('L4-S1椎体棘突', 244, 252, 'Bo'), ('压痛', 256, 257, 'Ch')]
1.患者中年男性,慢性发病,既往体健。2.患者缘于4天前在上班时搬重物不慎扭伤腰部,当时腰部活动受限,不敢用力咳嗽及打喷嚏,自行在家热敷治疗,疼痛未见明显缓解。近一天来腰痛伴左下肢疼痛症状加重,L3-S1椎间盘CT平扫示:腰椎生理曲度尚可,所示椎体边缘呈刺样增生,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,相应水平硬膜囊受压。3.入院查体、T:36.6℃、P:70次/分、R

# 3. Log Errors

In [16]:
L = ('两肺纹理', 229, 232, 'Bo')
A = ('两肺', 229, 230, 'Bo')

def match(L, A, cct):
    t1, start1, end1, e1 = L
    t2, start2, end2, e2 = A
    d = {}
    if set(range(start1, end1+1)).intersection(range(start2, end2+1)):
        idx = set(range(start1, end1+1)).union(range(start2, end2+1))
        # print()
        d['Text'] = cct._text[min(idx): max(idx) + 1]
        d['T_start'] = min(idx)
        d['T_end' ]  = max(idx) 
        d['Learned'] = t1
        d['LearnET'] = e1
        d['Annoted'] = t2
        d['AnnotET'] = e2
        d['FilePath']= cct.annotedFilePath
        return d
       
match(L, A, cct)

{'AnnotET': 'Bo',
 'Annoted': '两肺',
 'FilePath': '/Users/floyd/Desktop/Research/NER-CRF/cctner/dataset/annoted/batch1/病史特点/病史特点-193.txt',
 'LearnET': 'Bo',
 'Learned': '两肺纹理',
 'T_end': 232,
 'T_start': 229,
 'Text': '科检查:'}

In [17]:
unpaired = ('脓苔', 100, 101, 'Sy')
def matchUpair(unpaired, cct, kind):
    d = {}
    d['Text'], d['T_start'], d['T_end' ], e = unpaired
    d['FilePath']= cct.annotedFilePath
    if kind == 'L':
        d['Learned'], d['LearnET'] = d['Text'], e
        d['Annoted'], d['AnnotET'] = None, None
    else:
        d['Learned'], d['LearnET'] = None, None
        d['Annoted'], d['AnnotET'] = d['Text'], e
    return d

matchUpair(unpaired, cct, 'L')

{'AnnotET': None,
 'Annoted': None,
 'FilePath': '/Users/floyd/Desktop/Research/NER-CRF/cctner/dataset/annoted/batch1/病史特点/病史特点-193.txt',
 'LearnET': 'Sy',
 'Learned': '脓苔',
 'T_end': 101,
 'T_start': 100,
 'Text': '脓苔'}

In [18]:
#cct = cctTestList[100]
print('Only in Learned: ', [i for i in cct.learnedEntities if i not in cct.annotedEntities])
print('Only in Annoted: ', [i for i in cct.annotedEntities if i not in cct.learnedEntities])
print(cct._text)

Only in Learned:  [('腰部活动受限', 44, 49, 'Sy'), ('左下肢疼痛', 87, 91, 'Sy'), ('椎体边缘呈刺样增生', 122, 130, 'Sy'), ('腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘', 132, 156, 'Sy'), ('水平硬膜囊受压', 160, 166, 'Sy'), ('心肺腹', 218, 220, 'Bo'), ('异常', 223, 224, 'Sy'), ('S1椎体棘突两侧轻压痛', 247, 257, 'Sy'), ('放射', 260, 261, 'Sy'), ('双侧直腿抬高试验', 263, 270, 'Ch'), ('拾物试验', 276, 279, 'Ch')]
Only in Annoted:  [('腰部', 44, 45, 'Bo'), ('咳嗽', 55, 56, 'Sy'), ('疼痛', 71, 72, 'Sy'), ('左下肢', 87, 89, 'Bo'), ('疼痛', 90, 91, 'Sy'), ('L3-S1椎间盘CT', 97, 106, 'Ch'), ('椎体边缘', 122, 125, 'Bo'), ('腰4-腰5、腰5-骶1椎间盘', 132, 145, 'Bo'), ('椎体边缘', 153, 156, 'Bo'), ('硬膜囊', 162, 164, 'Bo'), ('查体', 172, 173, 'Ch'), ('R', 191, 191, 'Ch'), ('心', 218, 218, 'Bo'), ('肺', 219, 219, 'Bo'), ('腹', 220, 220, 'Bo'), ('腰椎', 233, 234, 'Bo'), ('L4-S1椎体棘突', 244, 252, 'Bo'), ('压痛', 256, 257, 'Ch')]
1.患者中年男性,慢性发病,既往体健。2.患者缘于4天前在上班时搬重物不慎扭伤腰部,当时腰部活动受限,不敢用力咳嗽及打喷嚏,自行在家热敷治疗,疼痛未见明显缓解。近一天来腰痛伴左下肢疼痛症状加重,L3-S1椎间盘CT平扫示:腰椎生理曲度尚可,所示椎体边缘呈刺样增生,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,相应水平硬膜囊受压。3.入院查体、T:36.6℃、P:70次/分、R

In [19]:
inter = list(set(cct.learnedEntities).intersection(set(cct.annotedEntities)))
OnlyLearned = [ i for i in cct.learnedEntities if i not in inter]   
OnlyAnnoted = [ i for i in cct.annotedEntities if i not in inter]

def logError(cct):
    log = []
    inter = list(set(cct.learnedEntities).intersection(set(cct.annotedEntities)))
    OnlyLearned = [ i for i in cct.learnedEntities if i not in inter]   
    OnlyAnnoted = [ i for i in cct.annotedEntities if i not in inter]
    
    pairedL = []
    pairedA = []
    for L in OnlyLearned:
        for A in OnlyAnnoted:
            d = match(L, A, cct)
            if d:
                log.append(d)
                pairedL.append(L)
                pairedA.append(A)
                
    for L in [i for i in OnlyLearned if i not in pairedL]:
        log.append(matchUpair(L, cct, 'L'))
        
    for A in [i for i in OnlyAnnoted if i not in pairedA]:
        log.append(matchUpair(A, cct, 'A'))
           
    if len(log) == 0:
        return pd.DataFrame()
    cols = ['FilePath', 'Text', 'T_start', 'T_end', 'Annoted', 'AnnotET', 'Learned', 'LearnET']
    return pd.DataFrame(log)[cols].sort_values('T_start')


a = logError(cct)
a.head(10)

,FilePath,Text,T_start,T_end,Annoted,AnnotET,Learned,LearnET
0,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,腰部活动受限,44,49,腰部,Bo,腰部活动受限,Sy
16,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,咳嗽,55,56,咳嗽,Sy,None,None
17,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,疼痛,71,72,疼痛,Sy,None,None
1,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,左下肢疼痛,87,91,左下肢,Bo,左下肢疼痛,Sy
2,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,左下肢疼痛,87,91,疼痛,Sy,左下肢疼痛,Sy
18,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,L3-S1椎间盘CT,97,106,L3-S1椎间盘CT,Ch,None,None
3,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,椎体边缘呈刺样增生,122,130,椎体边缘,Bo,椎体边缘呈刺样增生,Sy
4,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,132,156,腰4-腰5、腰5-骶1椎间盘,Bo,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,Sy
5,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,132,156,椎体边缘,Bo,腰4-腰5、腰5-骶1椎间盘局限性后凸超出椎体边缘,Sy
6,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,水平硬膜囊受压,160,166,硬膜囊,Bo,水平硬膜囊受压,Sy


In [20]:
log_list = []
for cct in cctTestList:
    a = logError(cct)
    if len(a) is not 0 :
        log_list.append(a)
LogError = pd.concat(log_list ).reset_index(drop = True)

error_log_path = 'demo/Evals/LogError_1abdp.csv'
LogError.to_csv(error_log_path, encoding='utf-8')
LogError.head(10)

,FilePath,Text,T_start,T_end,Annoted,AnnotET,Learned,LearnET
0,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,胀,0,0,胀,Sy,None,None
1,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,活动受限,2,5,None,None,活动受限,Sy
2,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,左膝摔伤后疼痛约,0,7,左膝,Bo,左膝摔伤后疼痛约,Sy
3,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,左膝摔伤后疼痛约,0,7,疼痛,Sy,左膝摔伤后疼痛约,Sy
4,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,承德市双滦区西地家属楼,5,15,None,None,承德市双滦区西地家属楼,Si
5,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,发热,30,31,发热,Sy,None,None
6,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,热性惊厥,72,75,热性惊厥,Si,None,None
7,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,河北省承德市宽城满族自治县宽城镇朝阳小区,9,28,None,None,河北省承德市宽城满族自治县宽城镇朝阳小区,Sy
8,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,急性化脓性扁桃体炎,71,79,急性化脓性扁桃体炎,Si,None,None
9,/Users/floyd/Desktop/Research/NER-CRF/cctner/d...,双滦区宝鼎花园,6,12,None,None,双滦区宝鼎花园,Sy


# 4. Calculate F1-Score

In [21]:
df = pd.DataFrame([cct.data() for cct in cctTestList])
df2 = df.T
df2

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Atoms,10,14,46,80,84,62,47,81,75,80,...,366,133,148,168,130,168,165,120,195,118
Bo_Annoted,0,1,0,0,0,0,0,0,0,0,...,26,9,7,10,4,13,6,9,14,13
Bo_Learned,0,0,0,0,0,0,0,0,0,0,...,8,3,3,3,2,6,2,2,6,8
Bo_Matched,0,0,0,0,0,0,0,0,0,0,...,7,3,3,2,2,5,2,2,6,7
Ch_Annoted,0,0,0,0,0,0,0,0,0,0,...,23,3,7,4,6,5,12,6,10,1
Ch_Learned,0,0,0,0,0,0,0,0,0,0,...,4,1,4,1,1,1,2,3,2,1
Ch_Matched,0,0,0,0,0,0,0,0,0,0,...,3,0,3,0,1,0,2,2,2,0
E_Annoted,1,2,1,4,3,4,3,5,4,4,...,65,17,21,21,18,28,26,21,37,24
E_Learned,1,1,1,3,3,4,1,5,3,4,...,38,15,17,17,10,22,17,15,23,17
E_Matched,0,0,1,2,2,3,1,4,1,1,...,18,5,11,4,7,11,10,8,14,11


In [22]:
Result = df.sum()
Result = Result.to_dict()
Result

{'Atoms': 65682,
 'Bo_Annoted': 2605,
 'Bo_Learned': 926,
 'Bo_Matched': 739,
 'Ch_Annoted': 2336,
 'Ch_Learned': 922,
 'Ch_Matched': 718,
 'E_Annoted': 7331,
 'E_Learned': 5226,
 'E_Matched': 2555,
 'Si_Annoted': 197,
 'Si_Learned': 262,
 'Si_Matched': 34,
 'Sy_Annoted': 1932,
 'Sy_Learned': 2977,
 'Sy_Matched': 1036,
 'Tr_Annoted': 261,
 'Tr_Learned': 134,
 'Tr_Matched': 28}

In [32]:
batch = cct.batch
list(batch['dataAnno']['fLabel'].values())

['Sy', 'Bo', 'Ch', 'Tr', 'Si']

In [27]:
entitiesLable = list(batch['dataAnno']['fLabel'].values()) + [ 'E']
resultList = []
for i in entitiesLable:
    d = dict()
    d['id'] = i
    for k in Result:
        if i in k:
            d[k.replace('_','').replace(i,'')] = Result[k]
    resultList.append(d)
    
resultList

[{'Annoted': 1932, 'Learned': 2977, 'Matched': 1036, 'id': 'Sy'},
 {'Annoted': 2605, 'Learned': 926, 'Matched': 739, 'id': 'Bo'},
 {'Annoted': 2336, 'Learned': 922, 'Matched': 718, 'id': 'Ch'},
 {'Annoted': 261, 'Learned': 134, 'Matched': 28, 'id': 'Tr'},
 {'Annoted': 197, 'Learned': 262, 'Matched': 34, 'id': 'Si'},
 {'Annoted': 7331, 'Learned': 5226, 'Matched': 2555, 'id': 'E'}]

In [28]:
R = pd.DataFrame(resultList)
R.set_index('id', inplace = True)
R.index.name = None
R

,Annoted,Learned,Matched
Sy,1932,2977,1036
Bo,2605,926,739
Ch,2336,922,718
Tr,261,134,28
Si,197,262,34
E,7331,5226,2555


In [29]:
R['R'] = R['Matched']/R['Annoted']
R['P'] = R['Matched']/R['Learned']
R['F1'] = 2*R['R']*R['P']/(R['R'] + R['P'])
R[['Annoted', 'Learned', 'Matched', 'R', 'P','F1']]

,Annoted,Learned,Matched,R,P,F1
Sy,1932,2977,1036,0.536232,0.348001,0.422082
Bo,2605,926,739,0.283685,0.798056,0.418578
Ch,2336,922,718,0.307363,0.778742,0.440761
Tr,261,134,28,0.107280,0.208955,0.141772
Si,197,262,34,0.172589,0.129771,0.148148
E,7331,5226,2555,0.348520,0.488902,0.406944


# 5. All in All

In [3]:
import sys
rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner/'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')

from splitresult import splitResult

#resultPath = tmpOutput

def match(L, A, cct):
    t1, start1, end1, e1 = L
    t2, start2, end2, e2 = A
    d = {}
    if set(range(start1, end1+1)).intersection(range(start2, end2+1)):
        idx = set(range(start1, end1+1)).union(range(start2, end2+1))
        # print()
        d['Text'] = cct._text[min(idx): max(idx) + 1]
        d['T_start'] = min(idx)
        d['T_end' ]  = max(idx) 
        d['Learned'] = t1
        d['LearnET'] = e1
        d['Annoted'] = t2
        d['AnnotET'] = e2
        d['FilePath']= cct.annotedFilePath
        return d
    
    
def matchUpair(unpaired, cct, kind):
    d = {}
    d['Text'], d['T_start'], d['T_end' ], e = unpaired
    d['FilePath']= cct.annotedFilePath
    if kind == 'L':
        d['Learned'], d['LearnET'] = d['Text'], e
        d['Annoted'], d['AnnotET'] = None, None
    else:
        d['Learned'], d['LearnET'] = None, None
        d['Annoted'], d['AnnotET'] = d['Text'], e
    return d


def logError(cct):
    log = []
    inter = list(set(cct.learnedEntities).intersection(set(cct.annotedEntities)))
    OnlyLearned = [ i for i in cct.learnedEntities if i not in inter]   
    OnlyAnnoted = [ i for i in cct.annotedEntities if i not in inter]
    
    pairedL = []
    pairedA = []
    for L in OnlyLearned:
        for A in OnlyAnnoted:
            d = match(L, A, cct)
            if d:
                log.append(d)
                pairedL.append(L)
                pairedA.append(A)
                
    for L in [i for i in OnlyLearned if i not in pairedL]:
        log.append(matchUpair(L, cct, 'L'))
        
    for A in [i for i in OnlyAnnoted if i not in pairedA]:
        log.append(matchUpair(A, cct, 'A'))
           
    if len(log) == 0:
        return pd.DataFrame()
    cols = ['FilePath', 'Text', 'T_start', 'T_end', 'Annoted', 'AnnotET', 'Learned', 'LearnET']
    return pd.DataFrame(log)[cols].sort_values('T_start')


def evalPerform(cctTest, resultPath, eType, cols, logPath = False):
    results = splitResult(resultPath)
    cctTestList = sum(cctTest, [])
    if len(cctTestList) != len(results):
        return "Bad Performance, Not Match!!!"
    for idx in range(len(cctTestList)):
        cct    = cctTestList[idx]
        result = results[idx]
        cct.outputCols = cols 
        cct.corpResult(result=result[1], newTag= eType) ## TODO
        cct.score = result[0]
        cct.getLearnedEntities(eType)
        
    if logPath:
        log_list = []
        for cct in cctTestList:
            a = logError(cct)
            if len(a) is not 0 :
                log_list.append(a)
        LogError = pd.concat(log_list).reset_index(drop = True)
        LogError.to_csv(logPath, encoding='utf-8')

    df = pd.DataFrame([cct.data() for cct in cctTestList])
    Result = df.sum()
    Result = Result.to_dict()
    
    List = []
    batch = cct.batch
    entitiesLable = list(batch['dataAnno']['fLabel'].values()) + ['E']
    # entitiesLable = ['Sy','Bo', 'Ch', 'Tr', 'Si'] + ['R'] + ['E']
    for i in entitiesLable:
        d = dict()
        d['id'] = i
        for k in Result:
            if i in k:
                d[k.replace('_','').replace(i,'')] = Result[k]
        List.append(d)
    
    R = pd.DataFrame(List)
    R.set_index('id', inplace = True)
    R.index.name = None

    R['R'] = R['Matched']/R['Annoted']
    R['P'] = R['Matched']/R['Learned']
    R['F1'] = 2*R['R']*R['P']/(R['R'] + R['P'])
    return R[['Annoted', 'Learned', 'Matched', 'R', 'P','F1']]

In [1]:
import sys
rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner/'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')

import pandas as pd
from loaddata import loadData, getTestData
from evals import evalPerform


# p = rootPath + 'pkldata/2017-11-03/R_CCT_Dict.p'
p = rootPath + 'pkldata/CCKS-2018-04-04/CCT_Dict.p'
cctTrain, cctTest = loadData(p, 10)

filePath = 'demo/test.txt'
cols = ['_atom', 'basicTag', 'dictTag', 'POSTag']
getTestData(cctTest, cols, filePath)


####################
from crftools import crf_test
# root = 'Users/floyd/Desktop/Research/NER-CRF/cctner/'
Input = filePath
Output = 'demo/Evals/result.txt'

import time
testPath = rootPath + 'crftools/crf_test'
inputPath = Input
outputPath = Output
modelPath = rootPath + 'models/1abdp/model'

btime = time.clock()
crf_test(crf_test_path=testPath,
         modelpath=modelPath,
         testfilepath=inputPath,
         resultpath=outputPath)
etime = time.clock()

print('Time Consumption: %03f s' % (etime - btime))
####################



#########TEST###########

resultPath = 'demo/Evals/result.txt'                                      ; print(cols)
logPath = 'demo/Evals/LogError_1abdp.csv'
evalPerform(cctTest, resultPath, 'LearnedETag', cols, logPath = logPath)

Time Consumption: 0.009117 s
['_atom', 'basicTag', 'dictTag', 'POSTag']


,Annoted,Learned,Matched,R,P,F1
Sy,1932,2977,1036,0.536232,0.348001,0.422082
Bo,2605,926,739,0.283685,0.798056,0.418578
Ch,2336,922,718,0.307363,0.778742,0.440761
Tr,261,134,28,0.107280,0.208955,0.141772
Si,197,262,34,0.172589,0.129771,0.148148
E,7331,5226,2555,0.348520,0.488902,0.406944
